# Pandas (Phần 2)

(Thư viện pandas)

In [64]:
import pandas as pd
import numpy as np

## Xử lý dữ liệu bị thiếu

pandas dùng các giá trị `None` hay `NaN` hay `NA` để đánh dấu **dữ liệu bị thiếu** (missing data). Trong đó, nên dùng nhất là `np.NaN`.

In [65]:
pd.Series([1, None, 2, pd.NA, 3, np.NaN])

0       1
1    None
2       2
3    <NA>
4       3
5     NaN
dtype: object

In [66]:
sr1 = pd.Series([1, 2, 3], index=list("abc"))
sr2 = pd.Series([4, 5, 6], index=list("bad"))
print(sr1)
print(sr2)
print(sr1 * sr2)

a    1
b    2
c    3
dtype: int64
b    4
a    5
d    6
dtype: int64
a    5.0
b    8.0
c    NaN
d    NaN
dtype: float64


Các thao tác trên bảng dữ liệu thường bỏ qua các giá trị NA (tương tự như cách Excel bỏ qua các ô trống).

In [127]:
data = pd.DataFrame([[1, 2, np.NaN],
                     [3, 4, np.NaN],
                     [5, np.NaN, np.NaN]])
data

,0,1,2
0,1,2.0,NaN
1,3,4.0,NaN
2,5,NaN,NaN


In [132]:
data.mean()

0    3.0
1    3.0
2    NaN
dtype: float64

In [133]:
data.mean(skipna=False)

0    3.0
1    NaN
2    NaN
dtype: float64

Ta dùng phương thức hàm `isna` (`isnull`) và `notna` (`notnull`) để kiểm tra giá trị nào là NA trong `Series` hay `DataFrame`. 

In [134]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [71]:
data.isna()

0    False
1     True
2    False
3     True
dtype: bool

In [136]:
data.notna()

0     True
1    False
2     True
3    False
dtype: bool

In [135]:
data[data.notna()]

0        1
2    hello
dtype: object

Ta dùng phương thức `dropna` để xóa NA (trong `Series`) hoặc dòng/cột chứa NA (trong `DataFrame`).

In [74]:
print(data)
print(data.dropna())

0        1
1      NaN
2    hello
3     None
dtype: object
0        1
2    hello
dtype: object


In [75]:
df = pd.DataFrame([[1, np.nan, 2],
                   [2, 3, 5],
                   [np.nan, 4, 6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [76]:
df.dropna() # xóa tất cả các dòng chứa NA

,0,1,2
1,2.0,3.0,5


In [77]:
df.dropna(axis=1) # xóa tất cả các cột chứa NA

,2
0,2
1,5
2,6


Ta dùng phương thức `fillna` để điền giá trị vào dữ liệu bị thiếu (thay NA bằng giá trị nào đó).

In [78]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [139]:
data.fillna(0) # thay NA bằng 0

0        1
1        0
2    hello
3        0
dtype: object

In [80]:
data.fillna(method='ffill') # thay NA bằng giá trị trước đó

/var/folders/lx/ljnxtnsx0bl2vz3157k1q3xr0000gn/T/ipykernel_43862/4189501623.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill') # thay NA bằng giá trị trước đó


a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [81]:
data.fillna(method='bfill') # thay NA bằng giá trị sau đó

/var/folders/lx/ljnxtnsx0bl2vz3157k1q3xr0000gn/T/ipykernel_43862/3770743375.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill') # thay NA bằng giá trị sau đó


a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [82]:
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [83]:
df.fillna(method='ffill', axis=1) # thay NA bằng giá trị trước đó trên cùng dòng

/var/folders/lx/ljnxtnsx0bl2vz3157k1q3xr0000gn/T/ipykernel_43862/4220772478.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=1) # thay NA bằng giá trị trước đó trên cùng dòng


,0,1,2
0,1.0,1.0,2.0
1,2.0,3.0,5.0
2,NaN,4.0,6.0


## Nhóm dữ liệu với `groupby`

Ta đã biết cách tổng hợp (aggregation) hay thu (reduction) dữ liệu trong pandas (và NumPy) với các hàm `sum`, `prod`, `mean`, `min`, `max` (hay `count`, `first`, `last`, `median`, `std`, `var`, `mad`).

In [140]:
ser = pd.Series(np.random.randint(0, 10, 5))
ser

0    7
1    1
2    3
3    8
4    9
dtype: int64

In [85]:
print(ser.sum())
print(ser.max())

27
8


In [86]:
mask = ser > 5
mask

0    False
1    False
2    False
3     True
4     True
dtype: bool

In [87]:
print(mask.sum())
print(mask.count())

2
5


In [141]:
df = pd.DataFrame({c: np.random.randint(0, 10, 5) for c in "ABC"})
df

,A,B,C
0,2,3,9
1,0,6,1
2,7,3,7
3,4,4,5
4,2,7,9


In [142]:
df.sum()

A    15
B    23
C    31
dtype: int64

In [143]:
df.sum(axis=1)

0    14
1     7
2    17
3    13
4    18
dtype: int64

Trước khi tổng hợp dữ liệu ta có thể thực hiện việc **tách nhóm** (split) với  `groupby`.

In [144]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'], 'data': range(1, 7)})
df

,key,data
0,A,1
1,B,2
2,C,3
3,A,4
4,B,5
5,C,6


In [145]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'], 'data': range(1, 7)})

for val, df in df.groupby("key"):
    print(f"Tách nhóm theo giá trị {val} trên key được DataFrame:")
    print(df)
    print(f"Có kết quả thu trên data là {df['data'].sum()}")
    print()


Tách nhóm theo giá trị A trên key được DataFrame:
  key  data
0   A     1
3   A     4
Có kết quả thu trên data là 5

Tách nhóm theo giá trị B trên key được DataFrame:
  key  data
1   B     2
4   B     5
Có kết quả thu trên data là 7

Tách nhóm theo giá trị C trên key được DataFrame:
  key  data
2   C     3
5   C     6
Có kết quả thu trên data là 9



Sau đó ta có thể thực hiện việc tổng hợp trên từng nhóm và **kết hợp** (combine) kết quả lại trong một `Series` hay `DataFrame`. 

In [147]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'], 'data': range(1, 7)})

df.groupby("key").sum()

,data
key,
A,5
B,7
C,9


Việc tách nhóm có thể được thực hiện trên giá trị của nhiều cột.

In [148]:
df = pd.DataFrame({'key1': ['A', 'A', 'A', 'B', 'B', 'B'],
                   'key2': ['a', 'a', 'b', 'a', 'b', 'b'],
                   'data1': range(1, 7),
                   'data2': range(11, 17)})
df

,key1,key2,data1,data2
0,A,a,1,11
1,A,a,2,12
2,A,b,3,13
3,B,a,4,14
4,B,b,5,15
5,B,b,6,16


In [149]:
df.groupby("key1").sum()

,key2,data1,data2
key1,,,
A,aab,6,36
B,abb,15,45


In [96]:
df.groupby("key2").sum()

,key1,data1,data2
key2,,,
a,AAB,7,37
b,ABB,14,44


In [150]:
df.groupby(["key1", "key2"]).sum()

data1  data2
key1 key2              
A    a         3     23
     b         3     13
B    a         4     14
     b        11     31

Việc tách nhóm cũng có thể dựa vào mảng các nhãn cho các dòng trong `DataFrame`.

In [151]:
df

,key1,key2,data1,data2
0,A,a,1,11
1,A,a,2,12
2,A,b,3,13
3,B,a,4,14
4,B,b,5,15
5,B,b,6,16


In [99]:
row_labels = [i % 2 for i in range(df.shape[0])]
row_labels

[0, 1, 0, 1, 0, 1]

In [100]:
df.groupby(row_labels).sum()

,key1,key2,data1,data2
0,AAB,abb,9,39
1,ABB,aab,12,42


Với `groupby` ta cũng có thể thay công việc thu trong "tách nhóm - thu - kết hợp" bằng **lọc** (filter), **biến đổi** (transform), **áp dụng** (apply). 

In [152]:
df

,key1,key2,data1,data2
0,A,a,1,11
1,A,a,2,12
2,A,b,3,13
3,B,a,4,14
4,B,b,5,15
5,B,b,6,16


In [102]:
df.groupby("key1").sum()   # thu với các hàm tổng hợp

,key2,data1,data2
key1,,,
A,aab,6,36
B,abb,15,45


In [103]:
df.groupby("key1").aggregate(['max', np.max, max]) # thu với hàm aggregate

/var/folders/lx/ljnxtnsx0bl2vz3157k1q3xr0000gn/T/ipykernel_43862/3319638965.py:1: FutureWarning: The provided callable <function max at 0x103751f70> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'max' instead.
  df.groupby("key1").aggregate(['max', np.max, max]) # thu với hàm aggregate
/var/folders/lx/ljnxtnsx0bl2vz3157k1q3xr0000gn/T/ipykernel_43862/3319638965.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'max' instead.
  df.groupby("key1").aggregate(['max', np.max, max]) # thu với hàm aggregate


key2         data1         data2        
      max max max   max max max   max max max
key1                                         
A       b   b   b     3   3   3    13  13  13
B       b   b   b     6   6   6    16  16  16

In [104]:
df.drop("key2", axis=1).groupby("key1").aggregate(['max', np.max, max])

/var/folders/lx/ljnxtnsx0bl2vz3157k1q3xr0000gn/T/ipykernel_43862/2409171835.py:1: FutureWarning: The provided callable <function max at 0x103751f70> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'max' instead.
  df.drop("key2", axis=1).groupby("key1").aggregate(['max', np.max, max])
/var/folders/lx/ljnxtnsx0bl2vz3157k1q3xr0000gn/T/ipykernel_43862/2409171835.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'max' instead.
  df.drop("key2", axis=1).groupby("key1").aggregate(['max', np.max, max])


data1         data2        
       max max max   max max max
key1                            
A        3   3   3    13  13  13
B        6   6   6    16  16  16

In [105]:
df.groupby('key1').aggregate({'data1': 'min', 'data2': 'max'})

,data1,data2
key1,,
A,1,13
B,4,16


In [106]:
df

,key1,key2,data1,data2
0,A,a,1,11
1,A,a,2,12
2,A,b,3,13
3,B,a,4,14
4,B,b,5,15
5,B,b,6,16


In [107]:
df.groupby("key1").sum()

,key2,data1,data2
key1,,,
A,aab,6,36
B,abb,15,45


In [108]:
df.groupby("key1").filter(lambda gr: gr["data1"].sum() > 10) # lọc với hàm filter

,key1,key2,data1,data2
3,B,a,4,14
4,B,b,5,15
5,B,b,6,16


In [109]:
df.drop("key2", axis=1).groupby("key1").transform(lambda gr: gr - gr.iloc[0])

,data1,data2
0,0,0
1,1,1
2,2,2
3,0,0
4,1,1
5,2,2


In [110]:
df.groupby("key1").apply(lambda gr: True if gr['data1'].sum() > 10 else False) # áp dụng với hàm apply

key1
A    False
B     True
dtype: bool

**Bài tập**

1. Nạp lại bảng dữ liệu `Iris Data Set` (https://archive.ics.uci.edu/ml/datasets/iris)

1. Kiểm tra để biết bảng dữ liệu này không có NA

1. Tính trung bình (mean) và độ lệch chuẩn (std) của từng đặc trưng (sepal length, sepal width, petal length, petal width) trên tất cả các hoa (tất cả các dòng dữ liệu)

1. Tính trung bình và độ lệch chuẩn của từng đặc trưng trên từng nhóm hoa phân theo nhãn lớp (class), từ đó đưa ra nhận xét

In [153]:
#1 Nạp lại bảng dữ liệu
iris_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
iris = pd.read_csv('iris.data', names=iris_names)
iris

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [157]:
#2. Kiểm tra để biết bảng dữ liệu này không có na
iris.isna().any()

sepal_length    False
sepal_width     False
petal_length    False
petal_width     False
class           False
dtype: bool

In [161]:

#3. Tính trung bình (mean) và độ lệch chuẩn (std) của từng đặc trưng (sepal length, sepal width, petal length, petal width) trên tất cả các hoa (tất cả các dòng dữ liệu)
mean_values = iris.mean(numeric_only=True)
std_values = iris.std(numeric_only=True)
print(f'{mean_values}', end='\n\n\n')
print(f'{std_values}')

sepal_length    5.843333
sepal_width     3.054000
petal_length    3.758667
petal_width     1.198667
dtype: float64


sepal_length    0.828066
sepal_width     0.433594
petal_length    1.764420
petal_width     0.763161
dtype: float64


## Bảng tổng hợp

Cũng như `groupby`, pandas hỗ trợ việc tách nhóm và tổng hợp dữ liệu tiện lợi với **bảng tổng hợp** (pivot table). Khác với `groupby`, các bảng tổng hợp được bố trí theo 2 chiều (thay vì 1 chiều).

In [111]:
df = pd.DataFrame({'key1': ['A', 'A', 'A', 'B', 'B', 'B'],
                   'key2': ['a', 'a', 'b', 'a', 'b', 'b'],
                   'data1': range(1, 7),
                   'data2': range(11, 17)})
df

,key1,key2,data1,data2
0,A,a,1,11
1,A,a,2,12
2,A,b,3,13
3,B,a,4,14
4,B,b,5,15
5,B,b,6,16


In [112]:
df.groupby(["key1", "key2"]).sum()

data1  data2
key1 key2              
A    a         3     23
     b         3     13
B    a         4     14
     b        11     31

In [113]:
df.groupby(["key1", "key2"]).sum().unstack()

data1     data2    
key2     a   b     a   b
key1                    
A        3   3    23  13
B        4  11    14  31

In [114]:
df.groupby(["key1", "key2"])["data1"].sum()

key1  key2
A     a        3
      b        3
B     a        4
      b       11
Name: data1, dtype: int64

In [115]:
df.groupby(["key1", "key2"])["data1"].sum().unstack()

key2,a,b
key1,,
A,3,3
B,4,11


In [116]:
df.pivot_table(values="data1", index="key1", columns="key2")

key2,a,b
key1,,
A,1.5,3.0
B,4.0,5.5


In [117]:
df.pivot_table(values="data1", index="key1", columns="key2").stack()

key1  key2
A     a       1.5
      b       3.0
B     a       4.0
      b       5.5
dtype: float64

In [118]:
df.pivot_table(values=["data1", "data1"], index="key1", columns="key2")

data1      data2      
key2     a    b     a     b
key1                       
A      1.5  3.0  11.5  13.0
B      4.0  5.5  14.0  15.5

In [119]:
df.pivot_table(values=["data1", "data1"], index="key1", columns="key2").stack()

data1  data2
key1 key2              
A    a       1.5   11.5
     b       3.0   13.0
B    a       4.0   14.0
     b       5.5   15.5

In [120]:
df.pivot_table(values=["data1", "data1"], index=["key1", "key2"])

data1  data2
key1 key2              
A    a       1.5   11.5
     b       3.0   13.0
B    a       4.0   14.0
     b       5.5   15.5

In [121]:
df.pivot_table(values=["data1", "data1"], columns=["key1", "key2"])

key1      A           B      
key2      a     b     a     b
data1   1.5   3.0   4.0   5.5
data2  11.5  13.0  14.0  15.5

In [122]:
df.pivot_table(values=["data1", "data1"], columns=["key1", "key2"]).T

data1  data2
key1 key2              
A    a       1.5   11.5
     b       3.0   13.0
B    a       4.0   14.0
     b       5.5   15.5

Phương thức `pivot_table` có nhiều lựa chọn khác nhau cho các tham số, tham khảo thêm https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html.

In [123]:
df

,key1,key2,data1,data2
0,A,a,1,11
1,A,a,2,12
2,A,b,3,13
3,B,a,4,14
4,B,b,5,15
5,B,b,6,16


In [124]:
df.pivot_table(index="key1", columns="key2", aggfunc=["sum", "max"])

sum                 max             
     data1     data2     data1    data2    
key2     a   b     a   b     a  b     a   b
key1                                       
A        3   3    23  13     2  3    12  13
B        4  11    14  31     4  6    14  16

In [125]:
df.pivot_table(index="key1", columns="key2", aggfunc={"data1": "sum", "data2": "max"})

data1     data2    
key2     a   b     a   b
key1                    
A        3   3    12  13
B        4  11    14  16

**Bài tập** Trên bảng dữ liệu `Iris Data Set`

1. Dùng pivot table để tính trung bình (mean) và độ lệch chuẩn (std) của từng đặc trưng (sepal length, sepal width, petal length, petal width) trên từng nhóm hoa phân theo nhãn lớp (class)